In [ ]:
import pandas as pd


: 

In [ ]:
books = pd.read_csv("booksummaries.txt", delimiter = "\t" )

: 

: 

In [ ]:
books.columns = ["WikiID", "FreebaseID", "Title", "Author", "PubDate", "OrgGenre", "OrgSummary"]

: 

In [ ]:
books.head()

: 

In [ ]:
books.info()

: 

In [ ]:
books.isnull().sum()

: 

In [ ]:
books.isna().sum()

: 

In [ ]:
books.WikiID.duplicated().sum()

: 

In [ ]:
books.Title.duplicated().sum()

: 

In [ ]:
books = books.drop(columns = "FreebaseID")

: 

In [ ]:
books_dup = books[books[["Title","Author"]].duplicated(keep = False)]
print(books_dup["Title"])
    

: 

In [ ]:
books.iloc[7083,5]

: 

In [ ]:
books.iloc[7087,5]

: 

for i in range(len(books)):
    for j in range(len(books)) :
        if ( books.iloc[i,books.columns.get_loc("Title")] == books.iloc[j,books.columns.get_loc("Title")]  and books.iloc[i,books.columns.get_loc("Author")] == books.iloc[j,books.columns.get_loc("Author")] ) :
            dup_with_author.append(books.iloc[i])
               pd.isna( books.iloc[2,books.columns.get_loc("Author")]) == True
              


In [ ]:
books.iloc[7087,books.columns.get_loc("Author")]

: 

In [ ]:
pd.isna( books.iloc[2,books.columns.get_loc("Author")])

: 

In [ ]:
df = pd.DataFrame()

: 

In [ ]:
df = pd.concat([df , books.iloc[[7087]] ])
df

: 

In [ ]:
df3 = books.iloc[[7087]]
df3

: 

In [ ]:
books.isnull().sum()

: 

In [ ]:
books[["Author", "PubDate", "OrgGenre"]].isnull().sum()

: 

In [ ]:
i = 0 
books_3null = pd.DataFrame()
for i in range(len(books)) :
    if  ( pd.isna( books.iloc[i,books.columns.get_loc("Author")]) and 
          pd.isna( books.iloc[i,books.columns.get_loc("PubDate")]) and 
          pd.isna( books.iloc[i,books.columns.get_loc("OrgGenre")]) ) :
        books_3null = pd.concat([books_3null , books.iloc[[i]] ] )

: 

In [ ]:
books_3null

: 

In [ ]:
# i will not remove this ,
# 1. ill find the CaculatedGenre from summary
# 2. ill find the Author and PubDate using webscraping .

: 

In [ ]:
# converting genre into clean ( originally its in json)

: 

In [ ]:
import json
test = books


: 

In [ ]:
test[["Author", "PubDate", "OrgGenre"]].isnull().sum()

: 

In [ ]:
test = test.fillna(" ")

: 

In [ ]:
test["OrgGenre"].values[3]

: 

In [ ]:
test[["Author", "PubDate", "OrgGenre"]].isnull().sum()

: 

In [ ]:
test["OrgGenre"].values[3]

: 

In [ ]:
test['OrgGenre'] = test['OrgGenre'].astype("string")

json.loads(test["OrgGenre"].values[3]).values()

: 

In [ ]:
clean = []
for i in range(len(test)):
    str4 = test["OrgGenre"].values[i]
    if str4 != ' ' :
        #clean = pd.concat(clean, json.loads(str4).values() )
        clean.append(list(json.loads(str4).values()))
    else :
        #clean = pd.concat(clean, " ")
        clean.append(" ")

: 

In [ ]:
# addig this column in our dataset 
books = books.drop(["OrgGenre"], axis = 1)


: 

In [ ]:
books.insert(4, "Genre", clean)

: 

In [ ]:
books.head()

: 

In [ ]:
#cleaning summary

: 

In [ ]:
import re


: 

In [ ]:
def clean_sum (text) :
    text = re.sub("\'", "", text)
    text = re.sub("^a-zA-Z", " ", text)
    text = ' '.join(text.split())
    text = text.lower()
    return text
    

: 

In [ ]:
test["Summary"] = test["OrgSummary"].apply(lambda x: clean_sum(x))

: 

In [ ]:
books["Summary"] = test["Summary"]

: 

In [ ]:
books.head()

: 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

: 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

: 

In [ ]:
from nltk.corpus import stopwords

: 

In [ ]:
df = books.copy()

: 

In [ ]:
df.fillna("--NotAvailable--")
df["Genre"].replace(" ", "--NotAvailable--")

: 

In [ ]:
df["GenreString"] = df["Genre"].apply(lambda x: ' '.join(x))

: 

: 

In [ ]:
df["combined_text"] = df["Summary"] + " " + df["Author"] + " " + df["GenreString"] 

: 

In [ ]:
tf = TfidfVectorizer(analyzer = "word", ngram_range = (1,2), min_df = 0, stop_words = 'english')

: 

In [ ]:
tfidf_matrix = tf.fit_transform(df['combined_text'].values.astype("str"))


: 

In [ ]:

tfidf_matrix

: 

In [ ]:
cosine = cosine_similarity(tfidf_matrix, tfidf_matrix)

: 

In [ ]:
def get_title_from_index(Index) :
    return df[df.Index == Index]["Title"].values[0]

def get_index_from_title(Title) :
    return df[df.Title == Title]["Index"].values[0]

def get_recommendations(book) :
    book_index = get_index_from_title(book)
    similar_books = list(enumerate(cosine[book_index]))
    sortedbooks = sorted(similar_books, key = lambda x:x[1] , reverse = True)[1:]
    i = 0
    for book in sortedbooks:
        print(get_title_from_index(book[0]) + " : " + df.Author[df["Index"] == book[0]] )
        i = i+1 
        if i > 10 :
            break
    

: 

In [ ]:
print(get_recommendations("The Stand"))

: 

In [ ]:
index = list(range(1,len(df)+1))

: 

In [ ]:
df.insert(0,"Index", index)

: 

In [ ]:
df.head()

: 

In [ ]:
i = 2231
df[df.Index == i]

: 

In [ ]:
df["combined_text"].isnull().sum()

: 

In [ ]:
print(get_recommendations("A Clockwork Orange"))

: 

In [ ]:
df.to_csv('rsdf.csv')

: 

In [ ]:
books.to_csv('rsbooks.csv')

: 

: 